In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

In [2]:
inventory = pd.read_csv("msom_inventory_data.csv", names = ('day','item_id','warehouse_id','warehouse_city_id','total_begin_qty','total_end_qty','Replen_in_qty','transfer_in_qty','sale_out_qty','transfer_out_qty'))
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13650723 entries, 0 to 13650722
Data columns (total 10 columns):
 #   Column             Dtype
---  ------             -----
 0   day                int64
 1   item_id            int64
 2   warehouse_id       int64
 3   warehouse_city_id  int64
 4   total_begin_qty    int64
 5   total_end_qty      int64
 6   Replen_in_qty      int64
 7   transfer_in_qty    int64
 8   sale_out_qty       int64
 9   transfer_out_qty   int64
dtypes: int64(10)
memory usage: 1.0 GB


In [3]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13650723 entries, 0 to 13650722
Data columns (total 10 columns):
 #   Column             Dtype
---  ------             -----
 0   day                int64
 1   item_id            int64
 2   warehouse_id       int64
 3   warehouse_city_id  int64
 4   total_begin_qty    int64
 5   total_end_qty      int64
 6   Replen_in_qty      int64
 7   transfer_in_qty    int64
 8   sale_out_qty       int64
 9   transfer_out_qty   int64
dtypes: int64(10)
memory usage: 1.0 GB


In [4]:
#What could be a better way of inventory allocation to improve customer satisfaction? 

inventory
#total_begin_qty: This column represents the total quantity of a particular item that is present in the warehouse at the beginning of a specific time period, such as a day, week, or month.

total_end_qty: This column represents the total quantity of the item that is present in the warehouse at the end of the same time period as total_begin_qty.

replen_in_qty: This column represents the quantity of the item that is replenished or restocked in the warehouse during the time period.

transfer_in_qty: This column represents the quantity of the item that is transferred into the warehouse from another location or warehouse during the time period.

sale_out_qty: This column represents the quantity of the item that is sold or shipped out of the warehouse to customers or other locations during the time period.

transfer_out_qty: This column represents the quantity of the item that is transferred out of the warehouse to another location or warehouse during the time period.

In [5]:
#timeseries on inventory

In [6]:
inventory.head()

,day,item_id,warehouse_id,warehouse_city_id,total_begin_qty,total_end_qty,Replen_in_qty,transfer_in_qty,sale_out_qty,transfer_out_qty
0,20170605,34474,88,108,0,0,0,0,0,0
1,20170426,34474,88,108,0,0,0,0,0,0
2,20170414,34474,88,108,0,0,0,0,0,0
3,20170427,34474,88,108,0,0,0,0,0,0
4,20170425,34474,88,108,0,0,0,0,0,0


In [7]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13650723 entries, 0 to 13650722
Data columns (total 10 columns):
 #   Column             Dtype
---  ------             -----
 0   day                int64
 1   item_id            int64
 2   warehouse_id       int64
 3   warehouse_city_id  int64
 4   total_begin_qty    int64
 5   total_end_qty      int64
 6   Replen_in_qty      int64
 7   transfer_in_qty    int64
 8   sale_out_qty       int64
 9   transfer_out_qty   int64
dtypes: int64(10)
memory usage: 1.0 GB


In [8]:
inventory['day'] = pd.to_datetime(inventory['day'], format='%Y%m%d')

In [9]:
#ARIMA

In [ ]:
df = inventory
#df = df[['day','total_end_qty','Replen_in_qty']]
df = df[['Replen_in_qty']]
print(df.head())
X = df.iloc[:, :].values

size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model = ARIMA(history, order=(0,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

#error = mean_squared_error(test, predictions)
#print('Test MSE: %.3f' % error)


In [ ]:
import pandas as pd
import statsmodels.api as sm

# Select the Replen_in_qty column as the endogenous variable
endog = inventory['Replen_in_qty']

# Fit a SARIMAX model to the data
model = sm.tsa.SARIMAX(endog=endog, order=(1,1,1), seasonal_order=(0,0,0,0))
results = model.fit()

# Print the model summary
print(results.summary())

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.95852D+00    |proj g|=  1.02838D-01


 This problem is unconstrained.



At iterate    5    f=  5.91621D+00    |proj g|=  1.06797D-02

At iterate   10    f=  5.90375D+00    |proj g|=  8.57917D-04

At iterate   15    f=  5.90352D+00    |proj g|=  3.05619D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     16     20      1     0     0   3.274D-07   5.904D+00
  F =   5.9035198027677342     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [ ]:
# from statsmodels.tsa.stattools import adfuller

# def adfuller_test(Replen_in_qty):
#     result = adfuller(Replen_in_qty)
#     labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations']
#     for value, label in zip(result, labels):
#         print(label + ': ' + str(value))

#     if result[1] <= 0.05:
#         print("Strong evidence against the null hypothesis (Ho), reject the null hypothesis. Data is stationary.")
#     else:
#         print("Weak evidence against null hypothesis, indicating it is non-stationary.")


# adfuller_test(inventory)

In [ ]:
# from statsmodels.tsa.stattools import adfuller  
# from numpy import log  
# import pandas as pd  
# res = adfuller( inventory_train.value.dropna())  
# print('Augmented Dickey-Fuller Statistic: %f' % res[0])  
# print('p-value: %f' % res[1])

In [ ]:
# model = ARIMA(inventory, order=(1,1,1))
# results = model.fit()

In [ ]:
# inventory_train = inventory[inventory['day'].dt.month < 3]
# inventory_test = inventory[inventory['day'].dt.month >= 5]

# print(inventory_train.shape, inventory_test.shape)

In [ ]:
# time_period = list(range(len(inventory_train)))

# inventory_train_lm = inventory_train.copy()
# inventory_train_lm['TimePeriod'] = time_period
# inventory_train_lm.tail()

In [ ]:
# import statsmodels.formula.api as smf

# lm = smf.ols(formula='total_begin_qty ~ day', data=inventory_train_lm).fit()

# print(lm.summary())